### Задание 1

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
from nltk.tokenize import sent_tokenize

In [2]:
dvach = open('2ch_corpus.txt', encoding='utf-8').read()

In [3]:
# Функция нормализации

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [4]:
norm_dvach = normalize(dvach)

In [5]:
# Превращение абсолютных частот в вероятности

vocab_dvach = Counter(norm_dvach)
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})

In [6]:
def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [7]:
sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_dvach_test = sentences_dvach[:5000]
sentences_dvach = sentences_dvach[5000:]

In [8]:
trigrams_dvach = Counter()
bigrams_dvach = Counter()
unigrams_dvach = Counter()


for sentence in sentences_dvach:
    trigrams_dvach.update(ngrammer(sentence,3))
    bigrams_dvach.update(ngrammer(sentence,2))
    unigrams_dvach.update(sentence)

In [9]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    trigrams_dvach.update(ngrammer(sentence))
    bigrams_dvach.update(ngrammer(sentence,2))
    unigrams_dvach.update(sentence)

In [10]:
from scipy.sparse import lil_matrix, csr_matrix

In [11]:
matrix_dvach = lil_matrix((len(bigrams_dvach), len(unigrams_dvach))) 
            
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {word:i for i, word in enumerate(id2bigram_dvach)}

for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2

    matrix_dvach[bigram2id_dvach[bigram],word2id_dvach[word3]] =  (trigrams_dvach[ngram]/bigrams_dvach[bigram])

In [12]:
matrix_dvach = csr_matrix(matrix_dvach)

In [13]:
np.random.choice(list(range(matrix_dvach.shape[1])), 
                 p=matrix_dvach[1].toarray()[0])

11030

In [14]:
def generate(matrix, id2bigram, id2word, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    for i in range(n):
        p = matrix[current_idx].toarray()[0]
        chosen = np.random.choice(list(range(matrix.shape[1])), p = p)
        text.append(id2word[chosen])
        if id2word[chosen] == '<end>':
            current_idx = bigram2id[start]
        else:
            part = id2bigram[current_idx] + ' ' + id2word[chosen]
            part = ' '.join(part.split()[1:])
            current_idx = bigram2id[part]
    
    return ' '.join(text)


In [15]:
print(generate(matrix_dvach, id2bigram_dvach, id2word_dvach, bigram2id_dvach, start='<start> <start>').replace('<end>', '\n'))

пострадавшей диагностированы сотрясение головного мозга не уверен поэтому спрашиваю 
 незнакомой ух блядь после такого вина подружились даже больше сделал уже были готовы к такому говноедству 
 топология произведения 
 а эти фото 
 ну ты чего конкретно хочешь 
 простой и быстрый 
 я так дико хочется предложить фаллоут 4 но есть бесплатные курсы говно ипама при успешном попадании можно выбить пилота и или инструктора или без кодинга 
 не проходишь на харде 
 да просто отдыхать 
 естественно пока она для бидла и саморазвивается анимой и комплухтерными играми живет на мельдониях 
 вопросов более не живу и счастье


In [16]:
print(generate(matrix_dvach, id2bigram_dvach, id2word_dvach, bigram2id_dvach, start='<start> <start>').replace('<end>', '\n'))

адмиралы а токитсуказе оставляют лишь ради айдла 
 колду лучше спирать 
 то есть умрёшь но я лучше воспользуюсь аналогией 
 крипово 
 а батонинг в теме на 4 
 третья последняя из известных хоккеистов на фото будто посрала пиздой ты ещё захочешь 
 именно из-за таких упырей как ты 
 лазерный луч летит со скоростью с в госконторе 2 года подарков судьбы не планируется может позже 
 ближайшие пол года-год 
 это никакой пользы не приносит хороший результат 
 с ума думаю уже с руководством типа 
 обоссал быдлана 
 мне важно что ты чувствовал когда сосал куну 



In [17]:
print(generate(matrix_dvach, id2bigram_dvach, id2word_dvach, bigram2id_dvach, start='<start> <start>').replace('<end>', '\n'))

да говно она как раз ту самую копеечку все заводские документы 
 потом переходите о тайнах подземной мск закажите с сайта вяжет кабеля не в школу и чувствуешь блядь как же я тебе запруфаю 
 какой закат если время в растрепанной одежде это тоже самое плюс правда удобнее доставать из кармана сотку и ездить на любых животных 13 инициатива он не в убыток 
 конечно карбидоборные и графитовые стержни деда похож на его месте предал бы почти всякий человек имеющий столько могущества ничего червепидорского как в малайзии пройти за геноцид ксеносов в ебенях космоса тоже есть свое мнение 
 к


In [18]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 

In [19]:
all_perplexity = []
for sent in sentences_dvach_test:
    probs = []
    for ngram in ngrammer(sent, 3):
        word1, word2, word3 = ngram.split()
        bigram = word1 + ' '+word2
        
        if ngram in trigrams_dvach and bigram in bigrams_dvach:
            probs.append(np.log(trigrams_dvach[ngram]/bigrams_dvach[bigram]))
        else:
            probs.append(np.log(0.00001))
    if perplexity(probs)!= np.inf: 
        all_perplexity.append(perplexity(probs))

<ipython-input-18-29b2627a57da>:5: RuntimeWarning: divide by zero encountered in double_scalars
  return p**(-1/N)


In [20]:
np.mean(all_perplexity)

31601.41487175953

### Задание 2

1. Для несловарных слов можно добавить псевдо-слово <UNK> в тестовую выборку. Есть два способа вычислить вероятности для модели с несловарными словами:

    1. Все несловарные слова в обучающей выборке сковертировать в токен <UNK> на этапе нормализации текста.
    2. Заменить все словарные слова на токен <UNK> на основе их частотности (когда нет предопределенного словаря).

2. Сглаживание — это метод равномерного распределения вероятностей для исключения нулевых вероятностей в необычных контекстах. Есть несколько типов алгоритмов, самый простой из которых, — это сглаживание Лапласа. Оно заключается в том, что прибавляется 1 к каждому числу n-gram'ов. 
Еще можно рассматривать сглаживание как снижение ненулевых вероятностей, чтобы добавить эти вероятности к нулевым. Этот алгоритм можно описать через частное сниженного числа n-gram'ов и изначального.